Parâmetros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_catalag_schema

Regras e Definições

In [0]:
%sql
select * from tiller.silver.tb_despesas_silver;

In [0]:
set_catalog_and_schema_gold() #Define catalog e schema conforme run

In [0]:
query = f"""
SELECT
    CAST(d.nuDeputadoId AS STRING) AS id_deputado,
    TRIM(d.sgPartido) AS sigla_partido,
    TRIM(d.sgUF) AS sigla_uf,
    CAST(d.ideDocumento AS STRING) AS id_despesa,
    TRIM(d.txtDescricao) AS tipo_despesa,
    REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(d.txtCNPJCPF, '[\\./-]', ''), '\\s+', ''), '[^0-9]', '') AS cnpj_cpf_fornecedor,
    TRIM(d.indTipoDocumento) AS tipo_documento,
    TRIM(d.txtNumero) AS numero_documento,
    CAST(d.datEmissao AS DATE) AS data_emissao,
    CAST(d.vlrDocumento AS DOUBLE) AS valor_documento,
    CAST(d.vlrGlosa AS DOUBLE) AS valor_glosa,
    CAST(d.vlrLiquido AS DOUBLE) AS valor_liquido,
    TRIM(d.urlDocumento) AS url_documento,
    d.numSubCota,
    COALESCE(d.txtDescricaoEspecificacao, 'Sem informação') AS descricao_especificacao,
    d.numParcela,
    d.numAno AS ano,
    d.numMes AS mes,
    d.fonte_arquivo,
    date_trunc('second', CURRENT_TIMESTAMP()) AS data_ingestao, -- Atualiza data de ingestão até o segundo apenas date_trunc
    date_trunc('second', CURRENT_TIMESTAMP()) AS data_modificacao
FROM tiller.silver.tb_despesas_silver d
LEFT JOIN tiller.gold.dim_deputados dep
    ON CAST(d.nuDeputadoId AS STRING) = dep.id_deputado
WHERE d.cpf IS NOT NULL
"""

df_despesas = spark.sql(query)

df_despesas.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("fato_despesas")

print("Fato criada/atualizada com sucesso!")


In [0]:
%sql

SELECT COUNT(*) AS total, MIN(data_ingestao) AS primeira_carga
FROM fato_despesas;


In [0]:
%sql
select * from tiller.gold.fato_despesas;